# **energyexplorer.io: data pipeline overview**

### **General overview of key steps:**

1: Get API key from NSRDB site and put in .gitignore \
2: Download radiation data from website and store as site_generation \
3: Run df through pySAM and export generation data to df - you should \
4: Download LMP data using script from CAISO_API.ipynb wrapper that Ranjit made \
5: Combine dataframes and calculate LCOE \

Use this tutorial for importing data: https://developer.nrel.gov/docs/solar/nsrdb/python-examples/

### **Directions for navigating notebook**

1: Follow the steps in each section to walk through the analysis performed to calculate the levelized cost of energy (LCOE) and net present value (NPV) of future cash flows for the user requested plant. \
2: Ignore sections of code with "SITE DEVELOPER ONLY" commented above them

### **Section 1. Get API Key from the National Renewable Energy Lab's (NREL) National Solar Radiation Database (NSRDB)**

#### **Section purpose**

This notebook remotely accesses solar radiation datasets using NSRDB's API. These datasets are used to calculate the annual solar radiation potential at a user defined latitude and longitude. 

To recieve data from the NSRDB's API, an API Key must be passed along with your data request.

#### **Directions for API**

1: Follow this link to sign up for an api key: https://developer.nrel.gov/signup/ (note: Data hosting and access is a free service provide by NREL, no finanical information is required to get your key.) \
2: When you have acquired your API Key replace text: "ENTER YOUR NSRDB API KEY HERE" with the API Key. Keep the quotes surrounding your API Key. \
3: Run code chunk.

In [10]:
###############################################################
# API Key to access National Solar Radiation Database (NSRDB) #
###############################################################

class creds:
    api_key = "ENTER YOUR NSRDB API KEY HERE"

##############################################################
# SITE DEVELOPER ONLY -- store API Key in creds.py ###########
# Import API Key via creds.py ################################
##############################################################

#import creds

### **Section 2. Import packages**

#### **Section purpose**

In this section, you import packages used by the data pipeline for data import an analysis. This interactive jupyter notebook is being hosted by a [JupyterLite](https://jupyterlite.readthedocs.io/en/latest/) server, a separate server from energyexplorer.io.

Because of this, we will need to install some packages that do not come natively with JupyterLite's kernel [Pyodide](https://pyodide.org/en/stable/). For a full list of packages included with JupyterLite, [click here](https://github.com/jupyterlite/jupyterlite/tree/main/packages).

In [3]:
#####################################################################################
# Pyodide non-included packages install #############################################
# SITE DEVELOPER ONLY -- comment out these installs as packages are already in venv #
#####################################################################################

%pip install NREL-PySAM
%pip install pycaiso
%pip install requests

########################################
# Pyodide non-included packages import #
########################################

# Import PySAM. This package provides python functions used to convert raw solar radiation values into energy generation based on the engineering parameters provided by the user
# Learn more about PySAM here: https://pypi.org/project/NREL-PySAM/

# Use site.addsitedir() to set the path to the SAM SDK API. Set path to the python directory.
import site
site.addsitedir('/Applications/sam-sdk-2015-6-30-r3/languages/python/')
import PySAM.PySSC as pssc

# Import pycasio. This package provides functions used to remotely access the California Independent System Operator's (CAISO) historical wholesale market price data.
# CAISO oversees the operation of California's bulk electric power system, transmission lines, and electricity market generated and transmitted by its member utilities.
# Learn more about CAISO here: https://www.caiso.com/Pages/default.aspx

import requests
from pycaiso.oasis import Node

########################################
# Pyodide included packages import #####
########################################

# General data wrangling

import pandas as pd
import numpy as np
import time
from datetime import datetime
from calendar import monthrange

# File management

import sys, os

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### 2. User parameter inputs are passed from home page model form to dictionary object

In [375]:
user_inputs = {
    'project_latitude': 35.21803686349634,
    'project_longitude': -116.94075390362501,
    'project_lifetime': 25,
    'project_type': 'solar',
    'installed_capacity': 100,
    'battery_capacity': 400,
    'max_charge_rate': 100,
    'max_discharge_rate': 100,
    'dc_ac_ratio': 1.1,
    'tilt': 25,
    'azimuth': 180,
    'inv_eff': 96,
    'losses': 14.0757,
    'array_type': 0,
    'gcr': 0.4,
    'adjust:constant': 0,
    'capital_cost_per_mwh': 890000, # example value used from https://www.nrel.gov/docs/fy22osti/80694.pdf - page 19
    'fixed_per_mwh': 16060, # example value used from https://www.nrel.gov/docs/fy22osti/80694.pdf
    'real_discount_rate': .05,
    'tax_credit': .30,
    'subsidy': 10
}

### 3. Pull historical solar radiation at requested lat, lon point from database

#### a. Prepare url for data import

Future development:
- need to decide what year of data should be pulled (or if multiple years should be pulled and averaged?)
- pull data once instead of twice

In [301]:
# Declare all variables as strings. Spaces must be replaced with '+', i.e., change 'John Smith' to 'John+Smith'.

# Define the lat, long of the location and the year
lat, lon, year = user_inputs['project_latitude'], user_inputs['project_longitude'], 2010

# lat = input() #testing input function
# Pull api key object from creds.py
api = creds.api_key

# Set the attributes to extract (e.g., dhi, ghi, etc.), separated by commas.
attributes = 'ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle'

# Choose year of data
year = '2020'

# Set leap year to true or false. True will return leap day data if present, false will not.
leap_year = 'false'

# Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
interval = '60'

# Specify Coordinated Universal Time (UTC), 'true' will use UTC, 'false' will use the local time zone of the data.
# NOTE: In order to use the NSRDB data in SAM, you must specify UTC as 'false'. SAM requires the data to be in the
# local time zone.
utc = 'false'

# Your full name, use '+' instead of spaces.
your_name = 'Tom+Wheeler'

# Your reason for using the NSRDB.
reason_for_use = 'research'

# Your affiliation
your_affiliation = 'UCSB'

# Your email address
your_email = 'tbwheeler@ucsb.edu'

# Please join our mailing list so we can keep you up-to-date on new developments.
mailing_list = 'false'

#### b. Pull dataset from api and save to pandas dataframe named df

Future development:
-need to conditionally check that dataframe always has 8760 rows

In [302]:
# create timezone and elevation objects for use in SAM model
# Declare url string
url = 'https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api, attr=attributes)

# Return just the first 2 lines to get metadata:
info = pd.read_csv(url, nrows=1)

# Assign timezone and elevation objects
timezone, elevation = info['Local Time Zone'], info['Elevation']

In [303]:
# Return all but first 2 lines of csv to get data:
site_generation = pd.read_csv('https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api, attr=attributes), skiprows=2)

# Set the time index in the pandas dataframe:
site_generation = site_generation.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))

# take a look
# print('shape:', site_generation.shape)
site_generation.head(25)

,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle
2020-01-01 00:00:00,2020,1,1,0,30,0,0,0,1.1,7.4,165.07
2020-01-01 01:00:00,2020,1,1,1,30,0,0,0,1.2,7.2,156.69
2020-01-01 02:00:00,2020,1,1,2,30,0,0,0,1.1,7.2,145.48
2020-01-01 03:00:00,2020,1,1,3,30,0,0,0,1.0,7.6,133.57
2020-01-01 04:00:00,2020,1,1,4,30,0,0,0,1.0,7.6,121.57
2020-01-01 05:00:00,2020,1,1,5,30,0,0,0,1.0,6.9,109.76
2020-01-01 06:00:00,2020,1,1,6,30,0,0,0,1.1,6.7,98.37
2020-01-01 07:00:00,2020,1,1,7,30,17,16,15,1.0,7.7,87.39
2020-01-01 08:00:00,2020,1,1,8,30,143,69,354,1.1,10.0,77.87
2020-01-01 09:00:00,2020,1,1,9,30,314,83,664,1.3,12.7,69.68


### 4. Import SAM simulation module to convert solar radiation data into energy output

In [304]:
ssc = pssc.PySSC()

# Resource inputs for SAM model:
# Must be byte strings
wfd = ssc.data_create()
ssc.data_set_number(wfd, b'lat', user_inputs['project_latitude'])
ssc.data_set_number(wfd, b'lon', user_inputs['project_longitude'])
ssc.data_set_number(wfd, b'tz', timezone)
ssc.data_set_number(wfd, b'elev', elevation)
ssc.data_set_array(wfd, b'year', site_generation.index.year)
ssc.data_set_array(wfd, b'month', site_generation.index.month)
ssc.data_set_array(wfd, b'day', site_generation.index.day)
ssc.data_set_array(wfd, b'hour', site_generation.index.hour)
ssc.data_set_array(wfd, b'minute', site_generation.index.minute)
ssc.data_set_array(wfd, b'dn', site_generation['DNI'])
ssc.data_set_array(wfd, b'df', site_generation['DHI'])
ssc.data_set_array(wfd, b'wspd', site_generation['Wind Speed'])
ssc.data_set_array(wfd, b'tdry', site_generation['Temperature'])

# Create SAM compliant object  
dat = ssc.data_create()
ssc.data_set_table(dat, b'solar_resource_data', wfd)
ssc.data_free(wfd)

# Specify the system Configuration
# Set system capacity in MW
# system_capacity = 4

ssc.data_set_number(dat, b'system_capacity', user_inputs['installed_capacity'])
# Set DC/AC ratio (or power ratio). See https://sam.nrel.gov/sites/default/files/content/virtual_conf_july_2013/07-sam-virtual-conference-2013-woodcock.psite_generation
ssc.data_set_number(dat, b'dc_ac_ratio', user_inputs['dc_ac_ratio'])
# Set tilt of system in degrees
ssc.data_set_number(dat, b'tilt', user_inputs['tilt'])
# Set azimuth angle (in degrees) from north (0 degrees)
ssc.data_set_number(dat, b'azimuth', user_inputs['azimuth'])
# Set the inverter efficency
ssc.data_set_number(dat, b'inv_eff', user_inputs['inv_eff'])
# Set the system losses, in percent
ssc.data_set_number(dat, b'losses', user_inputs['losses'])
# Specify fixed tilt system (0=Fixed, 1=Fixed Roof, 2=1 Axis Tracker, 3=Backtracted, 4=2 Axis Tracker)
ssc.data_set_number(dat, b'array_type', user_inputs['array_type'])
# Set ground coverage ratio
ssc.data_set_number(dat, b'gcr', user_inputs['gcr'])
# Set constant loss adjustment
ssc.data_set_number(dat, b'adjust:constant', user_inputs['adjust:constant'])

# execute and put generation results back into dataframe
mod = ssc.module_create(b'pvwattsv5')
ssc.module_exec(mod, dat)
site_generation['Generation (MW)'] = np.array(ssc.data_get_array(dat, b'gen'))

# free the memory
ssc.data_free(dat)
ssc.module_free(mod)

 0.00 %  @ 0
 0.67 %  @ 175
 1.34 %  @ 350
 2.00 %  @ 525
 2.67 %  @ 700
 3.33 %  @ 875
 4.00 %  @ 1050
 4.67 %  @ 1225
 5.33 %  @ 1400
 6.00 %  @ 1575
 6.66 %  @ 1750
 7.33 %  @ 1925
 7.99 %  @ 2100
 8.66 %  @ 2275
 9.33 %  @ 2450
 9.99 %  @ 2625
10.66 %  @ 2800
11.32 %  @ 2975
11.99 %  @ 3150
12.66 %  @ 3325
13.32 %  @ 3500
13.99 %  @ 3675
14.65 %  @ 3850
15.32 %  @ 4025
15.99 %  @ 4200
16.65 %  @ 4375
17.32 %  @ 4550
17.98 %  @ 4725
18.65 %  @ 4900
19.32 %  @ 5075
19.98 %  @ 5250
20.65 %  @ 5425
21.31 %  @ 5600
21.98 %  @ 5775
22.64 %  @ 5950
23.31 %  @ 6125
23.98 %  @ 6300
24.64 %  @ 6475
25.31 %  @ 6650
25.97 %  @ 6825
26.64 %  @ 7000
27.31 %  @ 7175
27.97 %  @ 7350
28.64 %  @ 7525
29.30 %  @ 7700
29.97 %  @ 7875
30.64 %  @ 8050
31.30 %  @ 8225
31.97 %  @ 8400
32.63 %  @ 8575
33.30 %  @ 8750


In [305]:
# Divide sum of generation by the number of periods times the system size
site_generation['Generation (MW)'].sum() / (525600/int(interval) * user_inputs['installed_capacity'])

0.1819632841125731

In [306]:
site_generation['Generation (MW)'].sum()

159399.83688261404

### 5. Find nearest node to user lat lon

- Full CAISO node dataset is from this repo: https://github.com/emunsing/CAISO-Scrapers
- Dataset xlsx is hosted here: https://github.com/emunsing/CAISO-Scrapers/blob/master/LMP%20Location%20Scraper/LMPLocations_vs_FullList.xls

Future development:
-Should Generation Nodes be filtered out? (i.e. show only load nodes)

In [307]:
nodes_locations = pd.read_csv('data/LMPLocations.csv')

nodes_locations.head()

,name,type,latitude,longitude
0,KERMAN_6_N001,Load Node,36.6921,-120.06
1,PINEFLT_2_B1,Generation Node,36.8400,-119.32
2,PINEFLT_7_B1,Generation Node,36.8300,-119.31
3,PINEFLT_7_B3,Generation Node,36.8300,-119.31
4,PINEFLT_7_B4,Generation Node,36.8300,-119.31


In [379]:
nodes_locations = pd.read_csv('data/LMPLocations.csv')

# find absolute value of user input lat/lon and node dataset lat/lon
# and subtract data lat/lon from user input lat/lons to get differences from user supplied lat lons
nodes_locations['latitude difference'] = abs(user_inputs['project_latitude']) - nodes_locations['latitude'].abs()
nodes_locations['longitude difference'] = abs(user_inputs['project_longitude']) - nodes_locations['longitude'].abs()

# add up absolute value of differences to find total difference then df by total difference to find location with the smallest total difference (the nearest node)
nodes_locations['total difference'] = nodes_locations['latitude difference'].abs() + nodes_locations['longitude difference'].abs() 
nodes_locations =nodes_locations.sort_values(by=['total difference'])

#extra desired node
node_request = nodes_locations['name'].iat[0]
node_request_lat = nodes_locations['latitude'].iat[0]
node_request_lon = nodes_locations['longitude'].iat[0]
print(node_request)

#show full dataframe
nodes_locations.head()

LUZ9G_7_B1


,name,type,latitude,longitude,latitude difference,longitude difference,total difference
2115,LUZ9G_7_B1,Generation Node,35.3500,-117.09,-0.131963,-0.149246,0.281209
2114,LUZ8G_7_B1,Generation Node,35.3400,-117.13,-0.121963,-0.189246,0.311209
2100,ALTA31GT_7_B1,Generation Node,34.8700,-116.88,0.348037,0.060754,0.408791
2098,ALTA40ST_7_B1,Generation Node,34.8700,-116.86,0.348037,0.080754,0.428791
1810,GALE_1_N001,Load Node,34.8595,-116.87,0.358537,0.070754,0.429291


### 6. Connect to CAISO API and pull down historical LMP data for nearest node

Note: Due to CAISO API timing out after only pulling 1 month of records (need to pull 12), the app temporarily pulls in a static dataset that was manually collected to complete the hw assignment back in Ranjit's class in February 2022. When API is working again, uncomment below code chunk, lmp dataset will be named the same for both the static dataset and the dataset pulled in via api.

See directory 'api reference/caiso-api.ipynb' for code that was used to build below code chunk (provided by Ranjit)

In [309]:
# # pull in historical node data from CAISO api
# # IMPORTANT: CAN ONLY BE PULLED IN 1 MONTH AT A TIME SO MUST RUN FOR LOOP

# # assign nearest node to desired node for import and use 2020 as import year. Caiso API does not support any earlier years (possibly in archives).
# node = Node(node_request)
# yr = 2020

# # initalize dataframe to append data imports from CAISO API
# node_lmps = pd.DataFrame()

# #change 3 to 12 to get full year
# for m in range(1,3):
#     print(m)
#     num_days = monthrange(yr, m)[1]
#     print(num_days)
#     node_lmps_month = node.get_month_lmps(yr, m)
# #     node_lmps_month = node.get_lmps(datetime(yr, m, num_days))
#     node_lmps = node_lmps.append(node_lmps_month, ignore_index=True)
#     time.sleep(5)
# #     node_lmps_month_last_day = node.get_lmps(datetime(yr, m, num_days))
# #     node_lmps = node_lmps.append(node_lmps_month_last_day, ignore_index=True)
# #     time.sleep(5)
#     print(node_lmps.head())

# #filter node_lmps output by only LMP
# lmp_only = node_lmps.query("LMP_TYPE == 'LMP'")

# lmp_only.head()

In [310]:
# Temporarily pull static dataset (2020 from HOLLISTR_1_N101 node)

lmp_only = pd.read_csv('data/LMP_Historic_price.csv')

# formatting dataframe to prepare for joining with power generation data

# replace T and miliseconds with blank and convert to datetime
lmp_only['INTERVALENDTIME_GMT'] = lmp_only['INTERVALENDTIME_GMT'].str.replace('T',' ')
lmp_only['INTERVALENDTIME_GMT'] = lmp_only['INTERVALENDTIME_GMT'].str.replace('-00:00','')
lmp_only['INTERVALENDTIME_GMT'] = pd.to_datetime(lmp_only['INTERVALENDTIME_GMT'], format="%Y-%m-%d %H:%M:%S")
lmp_only = lmp_only[['INTERVALENDTIME_GMT','OPR_HR', 'NODE', 'LMP_TYPE', 'MW']].sort_values(by='INTERVALENDTIME_GMT')

lmp_only.head()


,INTERVALENDTIME_GMT,OPR_HR,NODE,LMP_TYPE,MW
1,2020-01-01 09:00:00,1,HOLLISTR_1_N101,LMP,33.17687
2,2020-01-01 10:00:00,2,HOLLISTR_1_N101,LMP,30.89133
0,2020-01-01 11:00:00,3,HOLLISTR_1_N101,LMP,32.20839
3,2020-01-01 12:00:00,4,HOLLISTR_1_N101,LMP,31.34873
7,2020-01-01 13:00:00,5,HOLLISTR_1_N101,LMP,30.93713


### 7. Join site_generation (energy generation output at site) and lmp_only (historical marginal price at nearest wholesale node) to prepare for analysis

In [311]:
# Join generation and lmp dataframes on time

# drop datetime index and make into column named index
site_generation.reset_index(inplace=True)

In [314]:
site_generation

,index,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle,Generation (MW)
0,2020-01-01 00:00:00,2020,1,1,0,30,0,0,0,1.1,7.4,165.07,0.0
1,2020-01-01 01:00:00,2020,1,1,1,30,0,0,0,1.2,7.2,156.69,0.0
2,2020-01-01 02:00:00,2020,1,1,2,30,0,0,0,1.1,7.2,145.48,0.0
3,2020-01-01 03:00:00,2020,1,1,3,30,0,0,0,1.0,7.6,133.57,0.0
4,2020-01-01 04:00:00,2020,1,1,4,30,0,0,0,1.0,7.6,121.57,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2020-12-30 19:00:00,2020,12,31,19,30,0,0,0,1.2,7.7,120.63,0.0
8756,2020-12-30 20:00:00,2020,12,31,20,30,0,0,0,0.8,8.2,132.62,0.0
8757,2020-12-30 21:00:00,2020,12,31,21,30,0,0,0,0.7,8.2,144.55,0.0
8758,2020-12-30 22:00:00,2020,12,31,22,30,0,0,0,1.0,7.7,155.86,0.0


In [315]:
# rename time column to be be index so join can be performed using index column
lmp_only = lmp_only.rename(columns={'INTERVALENDTIME_GMT': 'index'})

# merge lmp_only and site_generation on index
# note: rows where date and time do not match will be dropped using inner join leading to a dataframe that is less than 8760 lines
lmp_generation_combined = pd.merge(site_generation, lmp_only, how='inner', on='index')

# print out the number of rows dropped to see how much mismatch there was between dfs
number_of_dropped_rows = 8760 - lmp_generation_combined.shape[0]
# print(f'{number_of_dropped_rows} rows were droped')

### 7. (Part 2) If project has a battery associated with it, build script to determine how their earnings would change

In [317]:
lmp_generation_combined = lmp_generation_combined[['index', 'Generation (MW)', 'NODE', 'MW']]
lmp_generation_combined

,index,Generation (MW),NODE,MW
0,2020-01-01 09:00:00,38.484135,HOLLISTR_1_N101,33.17687
1,2020-01-01 10:00:00,50.963184,HOLLISTR_1_N101,30.89133
2,2020-01-01 11:00:00,58.360352,HOLLISTR_1_N101,32.20839
3,2020-01-01 12:00:00,60.885323,HOLLISTR_1_N101,31.34873
4,2020-01-01 13:00:00,56.946399,HOLLISTR_1_N101,30.93713
...,...,...,...,...
8746,2020-12-30 19:00:00,0.000000,HOLLISTR_1_N101,31.51638
8747,2020-12-30 20:00:00,0.000000,HOLLISTR_1_N101,29.09134
8748,2020-12-30 21:00:00,0.000000,HOLLISTR_1_N101,28.17331
8749,2020-12-30 22:00:00,0.000000,HOLLISTR_1_N101,28.29448


In [370]:
### Problem statement ###
# 1. Must check that 

##########################################################
#### Create lists to store and test dataframe outputs ####
##########################################################

# with_battery_total = []

# without_battery_total = []

# difference_battery = []

all_dfs = []

##########################################################
###### Add date and hour values to columns in df #########
##########################################################

# create a new column using index column which has only the day, month, and year on each row
lmp_generation_combined['index_day_only'] = pd.to_datetime(lmp_generation_combined['index']).dt.date

# create a new column using index column which has only the time
lmp_generation_combined['index_hour_only'] = pd.to_datetime(lmp_generation_combined['index']).dt.hour

# split main dataframe into 365 subdataframes each with the same index_day_only value
daily_energy_dfs = {}
for idx, v in enumerate(lmp_generation_combined['index_day_only'].unique()):
    daily_energy_dfs[f'df{idx}'] = lmp_generation_combined.loc[lmp_generation_combined['index_day_only'] == v]

# loop through the list of 365 dataframes, 1 dataframe at a time
for day in np.arange(0, len(daily_energy_dfs), 1):
    
    # loop through the list of 365 dataframes, 1 dataframe at a time. assign current dataframe to current_df
    current_df = daily_energy_dfs[f'df{day}'].loc[:]

    # isolate df with only hours in the day where (1) there is no solar generation (2) the hour of the day is past 12 | and sort df with highest priced hours on top
    top_values_current_day = current_df[current_df['Generation (MW)'] == 0].sort_values('MW', ascending = False).query("index_hour_only > 12")

    # Sort by lowest day ahead price on top to highest price on bottom. We'll assume historical data is exactly representative of day ahead hourly prices.
    # Filter out non-producing hours as we assume battery system can only be filled by the solar system. 
    # These are the hours we'll want to be filling the battery.
    bot_values_current_day = current_df[current_df['Generation (MW)'] != 0].sort_values('MW')

    # Assign current battery charge to 0 at beginning of the day
    # Assume battery is discharged down to 1 at night (cannot be 0 or battery_discharge_hour_counter will not function properly)
    current_battery_charge = .00001

    # Calculate earnings per hour if plant had no battery
    current_df['earnings_per_hour'] = current_df['Generation (MW)'] * current_df['MW']

    # Iterate through each row in bot_values_current_day dataframe
    for hour in reversed(np.arange(0, len(bot_values_current_day), 1)):

        # Check if battery is below capacity
        # If battery is fully charged for the day, this loop will end
        if current_battery_charge < user_inputs['battery_capacity']:

            # calculate what hour in the evening the battery will discharge
            # for example: if the battery has been charged up to 70 MWh out of 400 MWh of capacity and the max discharge rate is 100 MW per hour then the current 70 MWh would be discharged in the first hour
            # for example: if the battery has been charged up to 130 MWh out of 400 MWh of capacity and the max discharge rate is 100 MW per hour then the first 100 MWs would be discharged in the first hour and the remaining 30 MWs in the second
            battery_discharge_hour_counter = int(np.ceil(current_battery_charge / user_inputs['max_discharge_rate'])) - 1

            # Conditionally check if the price per MW during the hour that this generation would be discharged to the grid is higher than the price that is currently being asking for
            if top_values_current_day['MW'].iat[battery_discharge_hour_counter] > bot_values_current_day['MW'].iat[hour]:

                # If battery is below capacity, conditionally decide how much to fill the battery

                # If total generation in that hour is more than max charging rate, then charge battery to max charge rate
                if bot_values_current_day['Generation (MW)'].iat[hour] > user_inputs['max_charge_rate']:

                    add_charge = user_inputs['max_charge_rate']

                # Else charge the battery with that hour's solar system generation (which should be less than or equal to user_inputs['max_charge_rate'])
                else:

                    add_charge = bot_values_current_day['Generation (MW)'].iat[hour]

                # Subtract the amount of charge added from the Generation for that hour
                bot_values_current_day['Generation (MW)'].iat[hour] = bot_values_current_day['Generation (MW)'].iat[hour] - add_charge

                # Update current battery charge with charge added
                current_battery_charge = current_battery_charge + add_charge

                # Check to see if battery capacity was exceeded (overflow charge) by adding new charge to current battery charge
                if current_battery_charge > user_inputs['battery_capacity']:

                    # Calculate overflow charge (the amount of excess charge 'sent' to the battery)
                    extra_charge = current_battery_charge - user_inputs['battery_capacity']

                    # Set current battery charge to battery_capacity by subtracting the extra charge from the total current_battery_charge
                    current_battery_charge = current_battery_charge - extra_charge

                    # Assign the overflow charge to be the generation sent to the grid for that hour. This will be the first hour on the bot_values_current_day dataframe that energy will be immediately sold to the grid
                    bot_values_current_day['Generation (MW)'].iat[hour] = extra_charge

        else:

            break
    

    #Overwrite current_df generation values with bot_values_current_day to reflect energy sent to battery instead of grid
    current_df = current_df.set_index('index')
    current_df.update(bot_values_current_day.set_index('index'))
    current_df = current_df.reset_index()

    ###############################################
    ### Calculate solar revenue for current day ###
    ###############################################

    # Calculate the earning per hour for solar only
    current_df['earnings_per_hour_solar'] = current_df['Generation (MW)'] * current_df['MW']

    # Assign total revenue for that day from solar as an object
    #solar_revenue_sum = bot_values_current_day['earnings_per_hour_solar'].sum()

    #################################################
    ### Calculate battery revenue for current day ###
    #################################################

    #update battery discharge counter again to account for last charge added to battery
    battery_discharge_hour_counter = int(np.ceil(current_battery_charge / user_inputs['max_discharge_rate']))

    add_charge_current_day = current_df[current_df['Generation (MW)'] == 0].sort_values('MW', ascending=False).query("index_hour_only > 12").nlargest(n= battery_discharge_hour_counter, columns = 'MW')
    
    if current_battery_charge > user_inputs['max_discharge_rate'] and current_battery_charge < user_inputs['battery_capacity']:

        add_charge_current_day['battery_discharge'] = user_inputs['max_discharge_rate']

        add_charge_current_day['battery_discharge'].iat[-1] = current_battery_charge % user_inputs['max_discharge_rate']

    elif current_battery_charge == user_inputs['battery_capacity']:

        add_charge_current_day['battery_discharge'] = user_inputs['max_discharge_rate']

    else:
        
        add_charge_current_day['battery_discharge'] = current_battery_charge
    
    add_charge_current_day['earnings_from_battery'] = add_charge_current_day['battery_discharge'] * add_charge_current_day['MW']

    # create final df with solar earnings and battery earnings by hour
    current_df = current_df.merge(add_charge_current_day[['index','battery_discharge','earnings_from_battery']], on='index', how='left')

    #compare totals
    # with_battery = current_df['earnings_from_battery'].sum() + current_df['earnings_per_hour_solar'].sum()
    # without_battery = current_df['earnings_per_hour'].sum()

    # #
    # with_battery_total.append(with_battery)
    # without_battery_total.append(without_battery)
    # difference_battery.append(with_battery - without_battery)
    all_dfs.append(current_df)

generation_with_battery = pd.concat(all_dfs).fillna(0)

generation_with_battery['earnings_from_battery_and_solar'] = generation_with_battery['earnings_per_hour_solar'] + generation_with_battery['earnings_from_battery']



In [374]:
generation_with_battery['earnings_from_battery_and_solar'].sum() - generation_with_battery['earnings_per_hour'].sum()

452470.36219729204

### Second attempt at battery script
- goal in this script is to check each row where energy is being generated against all future rows (including rows where energy is being generated and immediately sold to the grid) with the goal of discharging the battery during hours when the solar plant is producing energy and when it's not

In [330]:
### Problem statement ###
# 1. Must check that 

##########################################################
#### Create lists to store and test dataframe outputs ####
##########################################################

with_battery_total = []

without_battery_total = []

difference_battery = []

all_dfs = []

##########################################################
###### Add date and hour values to columns in df #########
##########################################################

# create a new column using index column which has only the day, month, and year on each row
lmp_generation_combined['index_day_only'] = pd.to_datetime(lmp_generation_combined['index']).dt.date

# create a new column using index column which has only the time
lmp_generation_combined['index_hour_only'] = pd.to_datetime(lmp_generation_combined['index']).dt.hour

# split main dataframe into 365 subdataframes each with the same index_day_only value
daily_energy_dfs = {}
for idx, v in enumerate(lmp_generation_combined['index_day_only'].unique()):
    daily_energy_dfs[f'df{idx}'] = lmp_generation_combined.loc[lmp_generation_combined['index_day_only'] == v]

# loop through the list of 365 dataframes, 1 dataframe at a time
for day in np.arange(0, len(daily_energy_dfs), 1):
    
    # loop through the list of 365 dataframes, 1 dataframe at a time. assign current dataframe to current_df
    current_df = daily_energy_dfs[f'df{day}'].loc[:]

    # Assign current battery charge to 0 at beginning of the day
    # Assume battery is discharged down to 1 at night (cannot be 0 or battery_discharge_hour_counter will not function properly)
    current_battery_charge = 1

    # Calculate earnings per hour if plant had no battery to use for comparison
    current_df['earnings_per_hour'] = current_df['Generation (MW)'] * current_df['MW']

    # Iterate through each row in current_df dataframe
    for hour in np.arange(0, len(current_df), 1):

        # Check if battery is below capacity
        # If battery is fully charged for the day, this loop will end
        if current_battery_charge < user_inputs['battery_capacity']:

            for sub_hour in np.arange(0, len(current_df), 1):

                # if price in current hour in first loop is less than the price in current hour in second loop AND the current hour in the first loop is less than the second loop (i.e. the hour in the day is further ahead)
                if current_df['MW'].iat[hour] < current_df['MW'].iat[sub_hour] and current_df['index_hour_only'].iat[hour] < current_df['index_hour_only'].iat[sub_hour]:

                    # If total generation in that hour is more than max charging rate, then charge battery to max charge rate
                    if bot_values_current_day['Generation (MW)'].iat[hour] > user_inputs['max_charge_rate']:
                    
                        add_charge = user_inputs['max_charge_rate']
    
                    # Else charge the battery with that hour's solar system generation (which should be less than or equal to user_inputs['max_charge_rate'])
                    else:
                    
                        add_charge = bot_values_current_day['Generation (MW)'].iat[hour]

                # Subtract the amount of charge added from the Generation for that hour
                bot_values_current_day['Generation (MW)'].iat[hour] = bot_values_current_day['Generation (MW)'].iat[hour] - add_charge

                # Update current battery charge with charge added
                current_battery_charge = current_battery_charge + add_charge

                # Check to see if battery capacity was exceeded (overflow charge) by adding new charge to current battery charge
                if current_battery_charge > user_inputs['battery_capacity']:

                    # Calculate overflow charge (the amount of excess charge 'sent' to the battery)
                    extra_charge = current_battery_charge - user_inputs['battery_capacity']

                    # Set current battery charge to battery_capacity by subtracting the extra charge from the total current_battery_charge
                    current_battery_charge = current_battery_charge - extra_charge

                    # Assign the overflow charge to be the generation sent to the grid for that hour. This will be the first hour on the bot_values_current_day dataframe that energy will be immediately sold to the grid
                    bot_values_current_day['Generation (MW)'].iat[hour] = extra_charge

        else:

            break
    

    #Overwrite current_df generation values with bot_values_current_day to reflect energy sent to battery instead of grid
    current_df = current_df.set_index('index')
    current_df.update(bot_values_current_day.set_index('index'))
    current_df = current_df.reset_index()

    ###############################################
    ### Calculate solar revenue for current day ###
    ###############################################

    # Calculate the earning per hour for solar only
    current_df['earnings_per_hour_solar'] = current_df['Generation (MW)'] * current_df['MW']

    # Assign total revenue for that day from solar as an object
    #solar_revenue_sum = bot_values_current_day['earnings_per_hour_solar'].sum()

    #################################################
    ### Calculate battery revenue for current day ###
    #################################################

    #update battery discharge counter again to account for last charge added to battery
    battery_discharge_hour_counter = int(np.ceil(current_battery_charge / user_inputs['max_discharge_rate']))

    add_charge_current_day = current_df[current_df['Generation (MW)'] == 0].sort_values('MW', ascending=False).query("index_hour_only > 12").nlargest(n= battery_discharge_hour_counter, columns = 'MW')
    
    if current_battery_charge > user_inputs['max_discharge_rate'] and current_battery_charge < user_inputs['battery_capacity']:

        add_charge_current_day['battery_discharge'] = user_inputs['max_discharge_rate']

        add_charge_current_day['battery_discharge'].iat[-1] = current_battery_charge % user_inputs['max_discharge_rate']

    elif current_battery_charge == user_inputs['battery_capacity']:

        add_charge_current_day['battery_discharge'] = user_inputs['max_discharge_rate']

    else:
        
        add_charge_current_day['battery_discharge'] = current_battery_charge
    
    add_charge_current_day['earnings_from_battery'] = add_charge_current_day['battery_discharge'] * add_charge_current_day['MW']

    # create final df with solar earnings and battery earnings by hour
    current_df = current_df.merge(add_charge_current_day[['index','battery_discharge','earnings_from_battery']], on='index', how='left')


    #compare totals
    with_battery = current_df['earnings_from_battery'].sum() + current_df['earnings_per_hour_solar'].sum()
    without_battery = current_df['earnings_per_hour'].sum()

    #
    with_battery_total.append(with_battery)
    without_battery_total.append(without_battery)
    difference_battery.append(with_battery - without_battery)
    all_dfs.append(current_df)



In [333]:
current_df

,index,Generation (MW),NODE,MW,index_day_only,index_hour_only,earnings_per_hour,earnings_per_hour_solar,battery_discharge,earnings_from_battery
0,2020-12-30 00:00:00,0.000000,HOLLISTR_1_N101,33.81799,2020-12-30,0.0,0.000000,0.000000,NaN,NaN
1,2020-12-30 01:00:00,0.000000,HOLLISTR_1_N101,44.89696,2020-12-30,1.0,0.000000,0.000000,NaN,NaN
2,2020-12-30 02:00:00,0.000000,HOLLISTR_1_N101,65.74948,2020-12-30,2.0,0.000000,0.000000,NaN,NaN
3,2020-12-30 03:00:00,0.000000,HOLLISTR_1_N101,60.41384,2020-12-30,3.0,0.000000,0.000000,NaN,NaN
4,2020-12-30 04:00:00,0.000000,HOLLISTR_1_N101,56.48496,2020-12-30,4.0,0.000000,0.000000,NaN,NaN
5,2020-12-30 05:00:00,0.000000,HOLLISTR_1_N101,53.67852,2020-12-30,5.0,0.000000,0.000000,NaN,NaN
6,2020-12-30 06:00:00,0.000000,HOLLISTR_1_N101,48.14335,2020-12-30,6.0,0.000000,0.000000,NaN,NaN
7,2020-12-30 07:00:00,0.000000,HOLLISTR_1_N101,45.51635,2020-12-30,7.0,0.000000,0.000000,NaN,NaN
8,2020-12-30 08:00:00,21.279381,HOLLISTR_1_N101,40.21256,2020-12-30,8.0,855.698373,855.698373,NaN,NaN
9,2020-12-30 09:00:00,0.000000,HOLLISTR_1_N101,33.69241,2020-12-30,9.0,1364.799579,0.000000,NaN,NaN


In [357]:
df_test = pd.DataFrame(columns=['generation','battery discharge', 'generation og', 'batt and generation diff'], index=np.arange(len(all_dfs)))

for i in np.arange(0, len(all_dfs), 1):
    df1 = all_dfs[i]
    df2 = daily_energy_dfs[f'df{i}']

    df_test['generation'][i] = df1['Generation (MW)'].sum()
    df_test['battery discharge'][i] = df1['battery_discharge'].sum()
    df_test['generation og'][i] = df2['Generation (MW)'].sum()
    df_test['batt and generation diff'][i] = (df1['Generation (MW)'].sum() + df1['battery_discharge'].sum()) - df2['Generation (MW)'].sum()

# print(df['Generation (MW)'].sum())
# print(df['battery_discharge'].sum())

In [363]:
df_test

,generation,battery discharge,generation og,batt and generation diff
0,316.714958,0.00001,316.714958,0.00001
1,42.86386,196.511623,239.375474,0.00001
2,106.860954,258.482982,365.343926,0.00001
3,368.452476,0.00001,368.452476,0.00001
4,192.39146,0.00001,192.39146,0.00001
...,...,...,...,...
360,192.0211,0.00001,192.0211,0.00001
361,167.33841,94.741638,262.080038,0.00001
362,39.896686,356.293638,396.190314,0.00001
363,253.794615,120.290928,374.085533,0.00001


### 8. Calculate key parameters to used for LCOE formula

Calculated paramaters are:
-Annual energy (year 1) = total_annual_generation
-DC capacity factor (year 1) = dc_capacity_factor
-CRF
-Operating Hours in Year (calculated by shaped of post-merge generation and price dataframes)

User supplied parameters (through form) are:
-Plant capacity
-Plant lifetime
-Real discount rate

In [26]:
annual_operating_hours = 8760 - number_of_dropped_rows

# subset columns to improve readability
lmp_generation_combined = lmp_generation_combined[['index', 'Generation (MW)', 'NODE', 'MW']]

# multiply generation per hour ['Generation (MW)'] by price per hour [MW] and put result in new column ['earnings_per_hour']
lmp_generation_combined['earnings_per_hour'] = lmp_generation_combined['Generation (MW)'] * lmp_generation_combined['MW']

# find sum of hourly generation and hourly earnings to find average earning per hour
total_annual_generation = lmp_generation_combined['Generation (MW)'].sum()
total_annual_earnings = lmp_generation_combined['earnings_per_hour'].sum()

# divide total annual earnings by total annual generation to find average earning per hour
average_earnings_per_hour = total_annual_earnings/total_annual_generation

#calculate capacity factor using equation (total actual generation by solar plant over course of year) / (installed_capacity * (number of hours in a year)) 
# note: number of hours in a year is reduced due to slight mismatch in power generation and the lmp hourly price dataset. when joining these two datasets, several hours of the year are dropped.
dc_capacity_factor = total_annual_generation / (user_inputs['installed_capacity'] * (8760 - number_of_dropped_rows))

# crf factor used to discount project cashflows over lifetime
crf = (user_inputs['real_discount_rate'] * (1 + user_inputs['real_discount_rate']) ** user_inputs['project_lifetime']) / ((1 + user_inputs['real_discount_rate']) ** user_inputs['project_lifetime'] - 1)

In [27]:
lcoe = (((user_inputs['capital_cost_per_mwh']*(1-user_inputs['tax_credit'])) * user_inputs['installed_capacity'] * crf) + (user_inputs['fixed_per_mwh'] * user_inputs['installed_capacity'])) / (user_inputs['installed_capacity'] * dc_capacity_factor * annual_operating_hours)

net_average_hourly_earnings = average_earnings_per_hour - lcoe

print(net_average_hourly_earnings)

-22.05266820506686


### 9. Build dataframe to calculate lifetime project value and graph


In [28]:
lifetime_value_df = pd.DataFrame()

selected_node = lmp_only['NODE'].iat[0]

lifetime_value_df['Year'] = np.arange(start= int(year), stop = int(year) + user_inputs['project_lifetime'], step= 1)
lifetime_value_df['Years from Start'] = np.arange(start= 0, stop = user_inputs['project_lifetime'], step= 1)
lifetime_value_df['LCOE ($/MWh)'] = round(lcoe, 2)
lifetime_value_df[f'Income from nearest node {selected_node} ($/MWh)'] = average_earnings_per_hour
lifetime_value_df['Inflation Adjusted Subsidy ($/MWh)'] = (user_inputs['subsidy']/((1+user_inputs['real_discount_rate'])**lifetime_value_df['Years from Start']))
lifetime_value_df['Subsidized Income ($/MWh)'] = round(lifetime_value_df[f'Income from nearest node {selected_node} ($/MWh)'] + lifetime_value_df['Inflation Adjusted Subsidy ($/MWh)'], 2)
lifetime_value_df['NPV Annual Income ($/Year)'] = round(((lifetime_value_df['Subsidized Income ($/MWh)'] - lifetime_value_df['LCOE ($/MWh)'])* total_annual_generation)/((1+user_inputs['real_discount_rate'])**lifetime_value_df['Years from Start']), 2)

project_lifetime_value = lifetime_value_df['NPV Annual Income ($/Year)'].sum()

lifetime_value_df

,Year,Years from Start,LCOE ($/MWh),Income from nearest node HOLLISTR_1_N101 ($/MWh),Inflation Adjusted Subsidy ($/MWh),Subsidized Income ($/MWh),NPV Annual Income ($/Year)
0,2020,0,49.7,27.644071,10.000000,37.64,-1922147.65
1,2021,1,49.7,27.644071,9.523810,37.17,-1901959.25
2,2022,2,49.7,27.644071,9.070295,36.71,-1877889.31
3,2023,3,49.7,27.644071,8.638376,36.28,-1847668.50
4,2024,4,49.7,27.644071,8.227025,35.87,-1813445.13
5,2025,5,49.7,27.644071,7.835262,35.48,-1775793.81
6,2026,6,49.7,27.644071,7.462154,35.11,-1735237.54
7,2027,7,49.7,27.644071,7.106813,34.75,-1693384.33
8,2028,8,49.7,27.644071,6.768394,34.41,-1649424.83
9,2029,9,49.7,27.644071,6.446089,34.09,-1603757.31


### 10. Build plot of project value


In [49]:
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go


#Create graph object Figure object with data
fig = go.Figure(data = go.Bar(x = lifetime_value_df.index, y = lifetime_value_df['NPV Annual Income ($/Year)'], marker_color='#0d6efd'))

#Update layout for graph object Figure
fig.update_layout(barmode='stack', 
                  title_text = f"NPV Annual Income ($/Year) At Lat: {user_inputs['project_latitude']}, Lon: {user_inputs['project_longitude']}",
                  xaxis_title = '($/Year)',
                  yaxis_title = 'Year',
                  yaxis_tickprefix = '$', 
                  yaxis_tickformat = ',')

fig

#plotly_plot_obj = plot({'data': fig}, output_type='div')


### 11. Build interactive plot

Follow this tutorial to build on previous plot and make it interactive so user can select multiple options